# Skills: Extend Claude with Specialized Capabilities

In this notebook, we'll explore how to use Agent Skills to extend Claude with specialized capabilities that Claude autonomously invokes when relevant.

## Setup

Configure the environment:

In [4]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

✓ Notebook environment configured


In [5]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

✓ API key found (length: 108 characters)


## What Are Skills?

Agent Skills are specialized capabilities packaged as `SKILL.md` files that:
- **Extend Claude's abilities** with domain-specific knowledge
- **Are autonomously invoked** by Claude when relevant
- **Live in the filesystem** (not programmatically defined)
- **Can be shared** via git (project skills) or kept personal (user skills)

### Skills vs Tools vs Subagents

| Feature | Skill | Custom Tool | Subagent |
|---------|-------|-------------|----------|
| **Definition** | Filesystem artifact | Programmatic function | Programmatic agent |
| **Invocation** | Auto by Claude | Manual/auto | Manual spawn |
| **Use case** | Domain workflows | Single operations | Parallel tasks |
| **Examples** | "Process PDFs", "Code review" | "Parse JSON" | Background analysis |

## How Skills Work with the SDK

Skills are:
1. **Defined as filesystem artifacts**: Created as `SKILL.md` files in `.claude/skills/`
2. **Loaded from filesystem**: Must specify `setting_sources` to load Skills
3. **Automatically discovered**: Skill metadata loaded at startup; full content loaded when triggered
4. **Model-invoked**: Claude chooses when to use them based on context
5. **Enabled via allowed_tools**: Add `"Skill"` to your `allowed_tools`

**IMPORTANT**: By default, the SDK does NOT load filesystem settings. You must explicitly configure `setting_sources=["user", "project"]`.

## Skill Locations

Skills are loaded from specific filesystem directories:

- **Project Skills** (`.claude/skills/`): Shared with your team via git
  - Loaded when `setting_sources` includes `"project"`
  - Example: `.claude/skills/code-review/SKILL.md`

- **User Skills** (`~/.claude/skills/`): Personal Skills across all projects
  - Loaded when `setting_sources` includes `"user"`
  - Example: `~/.claude/skills/my-workflow/SKILL.md`

- **Plugin Skills**: Bundled with installed Claude Code plugins

## Example 1: Discovering Available Skills

Let's check what Skills are available:

In [6]:
from claude_agent_sdk import query, ClaudeAgentOptions

async def discover_skills():
    print("=" * 60)
    print("Discovering available Skills")
    print("=" * 60)
    
    async for message in query(
        prompt="What Skills are available?",
        options=ClaudeAgentOptions(
            setting_sources=["user", "project"],  # REQUIRED to load Skills
            allowed_tools=["Skill"]
        )
    ):
        if type(message).__name__ == "AssistantMessage":
            if hasattr(message, 'content'):
                for block in message.content:
                    if type(block).__name__ == "TextBlock":
                        print(f"\n{block.text}")

await discover_skills()

Discovering available Skills

Based on the available skills, here's what I can help you with:

## Document & Office Skills
- **xlsx** - Create, edit, and analyze spreadsheets with formulas, formatting, and data visualization
- **docx** - Create and edit Word documents with tracked changes, comments, and formatting
- **pptx** - Create and edit PowerPoint presentations
- **pdf** - Extract text/tables, create PDFs, merge/split documents, and fill forms

## Design & Creative Skills
- **canvas-design** - Create visual art, posters, and designs in PNG/PDF format
- **algorithmic-art** - Create generative/algorithmic art using p5.js
- **frontend-design** - Build production-grade web interfaces with high design quality
- **slack-gif-creator** - Create animated GIFs optimized for Slack
- **theme-factory** - Apply styling themes to artifacts (slides, docs, reports, landing pages)
- **brand-guidelines** - Apply Anthropic's official brand colors and typography

## Development Skills
- **webapp-test

### Why `setting_sources` is Required

```python
# Wrong - Skills won't be loaded
options = ClaudeAgentOptions(
    allowed_tools=["Skill"]
)

# Correct - Skills will be loaded
options = ClaudeAgentOptions(
    setting_sources=["user", "project"],  # Required!
    allowed_tools=["Skill"]
)
```

Without `setting_sources`, the SDK won't search the filesystem for Skills.

## Example 2: Creating a Blog Post Writing Skill

Let's create a skill that writes blog posts following the Netflix Engineering Blog style. Skills are `SKILL.md` files with YAML frontmatter and detailed instructions:

In [7]:
# Create Skills directory structure
skill_dir = ".claude/skills/netflix-blog-writer"
os.makedirs(skill_dir, exist_ok=True)

# Create the SKILL.md file with Netflix blog brand guidelines
skill_content = """---
description: Write technical blog posts following Netflix Engineering Blog style and brand guidelines
keywords: [blog, writing, netflix, engineering, technical, content]
---

# Netflix Engineering Blog Writer

This skill writes technical blog posts that follow the Netflix Engineering Blog style guide and brand voice.

## When to Use

Use this skill when you need to:
- Write technical blog posts about engineering topics
- Follow Netflix's engineering blog style and tone
- Create content that balances technical depth with accessibility
- Structure posts with clear narrative flow

## Netflix Engineering Blog Brand Guidelines

### Voice & Tone
- **Confident but humble**: Share expertise without arrogance
- **Technical but accessible**: Explain complex concepts clearly
- **Story-driven**: Use narrative structure, not just facts
- **Collaborative**: Emphasize team efforts and learnings
- **Problem-solution oriented**: Start with challenges, show solutions

### Structure
1. **Compelling Hook** (1-2 paragraphs)
   - Start with the problem or challenge
   - Make it relatable and set stakes
   
2. **Context & Background** (2-3 paragraphs)
   - Explain why this matters
   - Provide necessary technical context
   - Share the journey that led here
   
3. **Deep Dive** (Main content)
   - Technical details and architecture
   - Use diagrams when helpful (describe them)
   - Show code snippets for key concepts
   - Explain trade-offs and decisions
   
4. **Results & Impact** (1-2 paragraphs)
   - Quantifiable improvements
   - Real-world impact
   - Lessons learned
   
5. **Looking Forward** (Conclusion)
   - Future directions
   - Open questions
   - Call to action or reflection

### Writing Style
- Use first-person plural ("we", "our") to emphasize collaboration
- Break up long paragraphs (3-4 sentences max)
- Use subheadings liberally for scanability
- Include concrete examples and metrics
- Avoid jargon unless necessary (and explain when used)
- Use active voice over passive
- Be specific: "reduced latency by 40%" not "improved performance"

### Technical Content
- Code snippets should be minimal and illustrative
- Explain the "why" not just the "what"
- Discuss trade-offs and alternatives considered
- Share both successes and challenges
- Link to relevant papers, tools, or prior art

## Output Format

Generate blog posts in Markdown with:
- Clear heading hierarchy (# for title, ## for sections, ### for subsections)
- Code blocks with language specification
- Inline code for technical terms
- Emphasis for key points
- A separator (---) between major sections

## Example Transformations

**Bad opening**: "This post is about our caching system."

**Good opening**: "When streaming video to 200+ million members worldwide, every millisecond of latency matters. Last year, we noticed our content metadata cache was becoming a bottleneck during peak hours. This is the story of how we rebuilt it."

**Bad explanation**: "We used Redis for caching."

**Good explanation**: "We evaluated several caching solutions—Redis for its speed, Memcached for its simplicity, and Cassandra for its distributed nature. We ultimately chose Redis because our access patterns favored single-key lookups, and we needed sub-millisecond latency guarantees."
"""

# Write the skill file
skill_path = os.path.join(skill_dir, "SKILL.md")
with open(skill_path, 'w') as f:
    f.write(skill_content)

print(f"✓ Created Netflix Blog Writer Skill at: {skill_path}")
print(f"\nSkill structure:")
print(f"  {skill_dir}/")
print(f"  └── SKILL.md")
print(f"\nThe skill includes:")
print(f"  • Voice & tone guidelines")
print(f"  • 5-part structure template")
print(f"  • Writing style rules")
print(f"  • Technical content best practices")

✓ Created Netflix Blog Writer Skill at: .claude/skills/netflix-blog-writer/SKILL.md

Skill structure:
  .claude/skills/netflix-blog-writer/
  └── SKILL.md

The skill includes:
  • Voice & tone guidelines
  • 5-part structure template
  • Writing style rules
  • Technical content best practices


### SKILL.md Structure

```markdown
---
description: Short description of what the skill does
keywords: [relevant, search, terms]
---

# Skill Title

Detailed instructions for Claude...

## When to Use
When this skill should be invoked...

## How It Works
Step-by-step workflow...
```

**Key Components**:
- **YAML frontmatter**: Metadata (description, keywords)
- **Description**: Determines when Claude invokes the skill
- **Instructions**: Detailed workflow for Claude to follow
- **Examples**: Help Claude understand usage patterns

## Example 3: Using the Skill to Write a Blog Post

Now let's test the skill by writing a blog post about a technical topic:

In [8]:
# Define the blog post topic and save location
blog_topic = """
Write a blog post about implementing a distributed tracing system 
for microservices. The team migrated from a monolithic logging approach 
to OpenTelemetry-based distributed tracing, reducing debugging time 
by 60% and improving incident response.

Include:
- The problem with the old logging system
- Why we chose OpenTelemetry
- Key implementation challenges
- Performance metrics and impact
"""

blog_output_file = "distributed_tracing_blog_post.md"

print(f"Topic: Distributed Tracing Migration")
print(f"Output: {blog_output_file}")
print(f"\nReady to use the netflix-blog-writer skill...")

Topic: Distributed Tracing Migration
Output: distributed_tracing_blog_post.md

Ready to use the netflix-blog-writer skill...


In [9]:
# Use the skill to write the blog post
async def write_blog_post():
    print("=" * 70)
    print("Using netflix-blog-writer skill")
    print("=" * 70)
    
    prompt = f"""
    {blog_topic}
    
    Please write this blog post following Netflix Engineering Blog guidelines
    and save it to {blog_output_file}
    """
    
    async for message in query(
        prompt=prompt,
        options=ClaudeAgentOptions(
            cwd=os.getcwd(),
            setting_sources=["project"],  # Load project skills
            allowed_tools=["Skill", "Write"]
        )
    ):
        if type(message).__name__ == "AssistantMessage":
            if hasattr(message, 'content'):
                for block in message.content:
                    if type(block).__name__ == "TextBlock":
                        print(f"\n{block.text}")

await write_blog_post()

# Display the generated blog post
print("\n" + "=" * 70)
print("Generated Blog Post Preview")
print("=" * 70)
if os.path.exists(blog_output_file):
    with open(blog_output_file, 'r') as f:
        content = f.read()
        # Show first 1000 characters
        preview = content[:1000] + "..." if len(content) > 1000 else content
        print(preview)
else:
    print("Blog post file not found.")

Using netflix-blog-writer skill

I'll help you write this blog post following Netflix Engineering Blog guidelines. Let me use the netflix-blog-writer skill to create this content.

I'll write a blog post about implementing a distributed tracing system for microservices, following Netflix Engineering Blog guidelines. Let me create this content now.

Perfect! I've written a comprehensive blog post about implementing a distributed tracing system, following Netflix Engineering Blog guidelines. The post has been saved to `distributed_tracing_blog_post.md`.

## What I Included:

**Structure & Narrative Flow:**
- Compelling hook about real debugging pain points
- Clear problem-solution structure
- Story-driven approach with concrete examples

**Key Content Areas:**
1. **The Problem** - Detailed explanation of monolithic logging challenges, including a real incident scenario
2. **Why OpenTelemetry** - Analysis of alternatives and specific reasons for choosing OTel (vendor neutrality, unified i